# Example modeling process

### Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations, 
                     get_training_labels, 
                     get_protein_proportions,
                     drop_empty_columns)

import pandas as pd
import numpy as np

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

### Load training data and labels

In [2]:
x_train_raw_counts = get_training_observations()
x_train = get_protein_proportions(x_train_raw_counts)
print(f"There are {x_train.shape[1]} features")
y_train = get_training_labels()

Getting all training observations from 'metagenome_classification.db'...
There are 16306 features
Getting all training labels from 'metagenome_classification.db'...


In [3]:
x_train.head(10)

index,PF00001.19,PF00002.22,PF00003.20,PF00004.27,PF00005.25,PF00006.23,PF00007.20,PF00008.25,PF00009.25,PF00010.24,...,PF17216.1,PF17217.1,PF17218.1,PF17219.1,PF17220.1,PF17221.1,PF17222.1,PF17223.1,PF17224.1,PF17225.1
0,0.000000e+00,0.000000e+00,0.0,0.004020,0.006243,0.001039,0.0,0.000000,0.003265,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000e+00,0.000000e+00,0.0,0.003575,0.013739,0.001026,0.0,0.000000,0.002235,0.000007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.376880e-07,2.188440e-07,0.0,0.001619,0.016218,0.000916,0.0,0.000002,0.001711,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000e+00,0.000000e+00,0.0,0.002737,0.019874,0.001785,0.0,0.000000,0.003570,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000e+00,0.000000e+00,0.0,0.001992,0.012389,0.001154,0.0,0.000013,0.002286,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000e+00,0.000000e+00,0.0,0.001684,0.016435,0.000931,0.0,0.000000,0.001806,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000e+00,0.000000e+00,0.0,0.001404,0.023274,0.000845,0.0,0.000000,0.001497,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000e+00,0.000000e+00,0.0,0.002301,0.015752,0.000925,0.0,0.000000,0.001843,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000e+00,0.000000e+00,0.0,0.001885,0.014797,0.001012,0.0,0.000000,0.001689,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000e+00,0.000000e+00,0.0,0.004574,0.013601,0.000995,0.0,0.000000,0.002402,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y_train.head()

index,EMPO_1,EMPO_2,EMPO_3
0,Free-living,Saline,Hypersaline (saline)
1,Free-living,Saline,Water (saline)
2,Host-associated,Plant,Plant rhizosphere
3,Free-living,Non-saline,Soil (non-saline)
4,Free-living,Saline,Water (saline)


### Data manipulation
Validation splits, dimensionality reduction, etc

In [5]:
# Split into train/validation if not CV
X_tr, X_val, Y_tr, Y_val = train_test_split(x_train, y_train, test_size=0.2) #, random_state=1)

# Dimensionality reduction?
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
svd.fit(X_tr)

new_x_train = svd.transform(X_tr)
new_x_val = svd.transform(X_val)


In [15]:
# convert string labels to numeric
labels3 = [
    'Aerosol (non-saline)',
    'Animal corpus',
    'Animal proximal gut',
    'Hypersaline (saline)',
    'Plant corpus',
    'Plant rhizosphere',
    'Plant surface',
    'Sediment (non-saline)',
    'Sediment (saline)',
    'Soil (non-saline)',
    'Subsurface (non-saline)',
    'Surface (non-saline)',
    'Surface (saline)',
    'Water (non-saline)',
    'Water (saline)'
]
labels3_map = {}
for i in range(0,len(labels3)):
    label = labels3[i]
    labels3_map[label] = i

Y_tr['EMPO_3_int'] = Y_tr['EMPO_3'].map(labels3_map)
Y_tr.head()
Y_val['EMPO_3_int'] = Y_val['EMPO_3'].map(labels3_map)

### Model training

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

def build_model(n_classes,
                hidden_layer_sizes=[],
                activation='relu',
                final_layer_activation='softmax',
                dropout=0.0,
                optimizer='Adam',
                learning_rate=0.01):
  """Build a multi-class logistic regression model using Keras.

  Args:
    n_classes: Number of output classes in the dataset.
    hidden_layer_sizes: A list with the number of units in each hidden layer.
    activation: The activation function to use for the hidden layers.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The desired learning rate for the optimizer.

  Returns:
    model: A tf.keras model (graph).
  """
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)
  model = keras.Sequential()
  model.add(keras.layers.Flatten())

  for hidden_layer_size in hidden_layer_sizes:
    model.add(keras.layers.Dense(hidden_layer_size))
    model.add(keras.layers.Activation(activation))
    if dropout > 0:
      model.add(keras.layers.Dropout(dropout))

  model.add(keras.layers.Dense(n_classes))
  model.add(keras.layers.Activation(final_layer_activation))
  opt = None
  if optimizer == 'SGD':
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
  elif optimizer == 'Adam':
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  else:
    raise f"Unsupported optimizer, {optimizer}"
  model.compile(loss='sparse_categorical_crossentropy', 
    optimizer=opt, metrics=['accuracy'])    
  return model

def train_model(X_train, Y_train, num_classes,
                       hidden_layer_sizes=[],
                       activation='tanh',
                       final_layer_activation='softmax',
                       dropout=0.2,
                       optimizer='Adam',
                       learning_rate=0.01,
                       batch_size=64,
                       num_epochs=5):

  # Build the model.
  model = build_model(num_classes,
                      hidden_layer_sizes=hidden_layer_sizes,
                      activation=activation,
                      final_layer_activation=final_layer_activation,
                      dropout=dropout,
                      optimizer=optimizer,
                      learning_rate=learning_rate)

  # Train the model.
  print('Training...')
  history = model.fit(
    x=X_train,
    y=Y_train,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_split=0.1,
    verbose=1)

  model.summary()
  return model

nn3 = train_model(X_tr, Y_tr['EMPO_3_int'], 15,
    hidden_layer_sizes=[256, 256],
    dropout=0.2,
    optimizer='Adam',
    learning_rate=0.01,
    batch_size=128,
    num_epochs=100)


Training...
Epoch 1/100
8/8 [==============================] - 1s 51ms/step - loss: 2.2158 - accuracy: 0.2670 - val_loss: 2.2361 - val_accuracy: 0.1700
Epoch 2/100
8/8 [==============================] - 0s 40ms/step - loss: 1.7947 - accuracy: 0.3715 - val_loss: 1.7115 - val_accuracy: 0.4100
Epoch 3/100
8/8 [==============================] - 0s 37ms/step - loss: 1.3975 - accuracy: 0.5128 - val_loss: 1.5854 - val_accuracy: 0.6000
Epoch 4/100
8/8 [==============================] - 0s 36ms/step - loss: 1.3142 - accuracy: 0.5695 - val_loss: 1.1378 - val_accuracy: 0.7000
Epoch 5/100
8/8 [==============================] - 0s 51ms/step - loss: 1.0229 - accuracy: 0.6974 - val_loss: 1.1477 - val_accuracy: 0.7200
Epoch 6/100
8/8 [==============================] - 0s 52ms/step - loss: 0.9939 - accuracy: 0.6986 - val_loss: 0.9005 - val_accuracy: 0.7600
Epoch 7/100
8/8 [==============================] - 0s 43ms/step - loss: 0.8538 - accuracy: 0.7319 - val_loss: 0.7325 - val_accuracy: 0.7800
Epoch 8/

8/8 [==============================] - 0s 39ms/step - loss: 0.2422 - accuracy: 0.9210 - val_loss: 0.5508 - val_accuracy: 0.8900
Epoch 60/100
8/8 [==============================] - 0s 39ms/step - loss: 0.1691 - accuracy: 0.9433 - val_loss: 0.4714 - val_accuracy: 0.9000
Epoch 61/100
8/8 [==============================] - 0s 36ms/step - loss: 0.1176 - accuracy: 0.9588 - val_loss: 0.4723 - val_accuracy: 0.9100
Epoch 62/100
8/8 [==============================] - 0s 39ms/step - loss: 0.1123 - accuracy: 0.9611 - val_loss: 0.4344 - val_accuracy: 0.9100
Epoch 63/100
8/8 [==============================] - 0s 33ms/step - loss: 0.1006 - accuracy: 0.9644 - val_loss: 0.4683 - val_accuracy: 0.9200
Epoch 64/100
8/8 [==============================] - 0s 35ms/step - loss: 0.1000 - accuracy: 0.9600 - val_loss: 0.4387 - val_accuracy: 0.9100
Epoch 65/100
8/8 [==============================] - 0s 34ms/step - loss: 0.0944 - accuracy: 0.9700 - val_loss: 0.5332 - val_accuracy: 0.9300
Epoch 66/100
8/8 [========

### Model evaluation

In [27]:
# Scoring model

nn3_accuracy = nn3.evaluate(x=X_val, y=Y_val['EMPO_3_int'], verbose=0, return_dict=True)['accuracy']
print(f"Accuracy on EMPO 3: {nn3_accuracy}")

Accuracy on EMPO 3: 0.9359999895095825


### Retrain best model
After experimenting with models, retrain your favorite model using entire training set (including validation) before saving

In [ ]:
import time

start_time = time.time()

#clf = LogisticRegression(multi_class='multinomial').fit(x_train, y_train['EMPO_1'])

end_time = time.time()
wallclock = int(end_time - start_time)
print(f"Wallclock = {wallclock} sec")

### Save fitted model

In [ ]:
# Save best model as joblib or pkl file to 'model_joblibs' folder
from joblib import dump

dump(clf, '../model_joblibs/example_neural_network.joblib')